In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Getting data

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
def plot_stats(feature,label_rotation=False,horizontal_layout=True):
    temp = df[feature].value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Number of contracts': temp.values})

    # Calculate the percentage of target=1 per category value
    cat_perc = df[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)
    
    if(horizontal_layout):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,14))
    sns.set_color_codes("pastel")
    s = sns.barplot(ax=ax1, x = feature, y="Number of contracts",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    s = sns.barplot(ax=ax2, x = feature, y='TARGET', order=cat_perc[feature], data=cat_perc)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.ylabel('Percent of target with value 1 [%]', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=10)

    plt.show();

In [ ]:
def plot_distribution_comp(var,nrow=2):
    
    i = 0
    t1 = df.loc[df['TARGET'] != 0]
    t0 = df.loc[df['TARGET'] == 0]

    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(nrow,2,figsize=(12,6*nrow))

    for feature in var:
        i += 1
        plt.subplot(nrow,2,i)
        sns.kdeplot(t1[feature], bw=0.5,label="TARGET = 1")
        sns.kdeplot(t0[feature], bw=0.5,label="TARGET = 0")
        plt.ylabel('Density plot', fontsize=12)
        plt.xlabel(feature, fontsize=12)
        locs, labels = plt.xticks()
        plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show();

In [ ]:
df = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test =  pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
df.head()

In [ ]:
missing_values_table(df)

In [ ]:
df.dtypes.value_counts()

# EDA

In [ ]:
df['TARGET'].value_counts()

data is imbalanced

In [ ]:
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

Most of the categorical variables are having 2 unique variables so its ok to use label encoding with columns which has 2 unique variables and one hot encoding with the rest.

In [ ]:
df.select_dtypes(['float64','int64']).describe()

let's explore distribution for some appealing columns

In [ ]:
df.head()

In [ ]:
df.columns.tolist()

In [ ]:
plot_stats('CODE_GENDER')

The number of female clients is almost double the number of male clients. Looking to the percent of defaulted credits, males have a higher chance of not returning their loans.

also there's a strange value 'XNA' , you will need to remove later

In [ ]:
plot_stats('FLAG_OWN_CAR')

People who doesn't own a car are more likely to not return their loan

In [ ]:
plot_stats('FLAG_OWN_REALTY')

This feature is not actually interesting as it contibute the same towards return/not return the loan

In [ ]:
plot_stats('CNT_CHILDREN')

So more childern contibute alot in the way on not returning the loans

In [ ]:
plot_stats('NAME_INCOME_TYPE',True)

It seems that both unemployed and people who take Maternity leave are struggling in returning their loans, we can add a flag Unemployed/Maternity leave later in feature engineering part

In [ ]:
plot_stats('NAME_FAMILY_STATUS',True)

We can try to merge both Civil marriage and Married under on variable later

In [ ]:
plot_stats('NAME_HOUSING_TYPE',True)

Both people with rented apartment and who lives with their parents are more likely to struggle in returning the loan

In [ ]:
plot_stats('OCCUPATION_TYPE',True)

low skill laborers flag can be added 

In [ ]:
plot_stats('CNT_FAM_MEMBERS',True)

More family memembers contibutes alot towards non returning of loans

In [ ]:
plot_stats('REG_CITY_NOT_LIVE_CITY')
plot_stats('REG_CITY_NOT_WORK_CITY')

### Let's explore some continuous features aganist their target

In [ ]:
var = ['AMT_ANNUITY','AMT_GOODS_PRICE','DAYS_EMPLOYED', 'DAYS_REGISTRATION','DAYS_BIRTH','DAYS_ID_PUBLISH']
plot_distribution_comp(var,nrow=3)

Those feature are somehow equally distributed for target = 0 or target =1

## Anomality handling

In [ ]:
df['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
df['CNT_CHILDREN'].plot.hist(title = 'CNT_CHILDREN');
plt.xlabel('CNT_CHILDREN');

In [ ]:
df['AMT_ANNUITY'].plot.hist(title = 'AMT_ANNUITY');
plt.xlabel('AMT_ANNUITY');

In [ ]:
df['CNT_FAM_MEMBERS'].plot.hist(title = 'CNT_FAM_MEMBERS');
plt.xlabel('CNT_FAM_MEMBERS');

In [ ]:
df = df[df['DAYS_EMPLOYED'] != 365243]
df = df[df['CNT_CHILDREN']<4]
df = df[df['AMT_ANNUITY']<120000]
df = df[df['CNT_FAM_MEMBERS']<7.5]




# Correlations

In [ ]:
correlations = df.corr()['TARGET'].sort_values()
correlations

strong negative correlation 

# Feature engineering

In [ ]:
df.NAME_INCOME_TYPE.value_counts()

In [ ]:
df = df.append(test)
df['DAYS_BIRTH'] = abs(df['DAYS_BIRTH'])
df['family_members_more7']= np.where(df['CNT_FAM_MEMBERS']>7,1,0)
df['islowskilled_labour']= np.where(df['OCCUPATION_TYPE']=='Low-skill Laborers',1,0)
df['is_Maternity_leave']= np.where(df['NAME_INCOME_TYPE'] =='Maternity leave' ,1,0)
df['is_unemployed']= np.where(df['NAME_INCOME_TYPE'] =='Unemployed' ,1,0)

df['cnt_childern_more6']= np.where(df['CNT_CHILDREN'] > 6,1,0)
plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(df['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

In [ ]:
df.family_members_more7.value_counts()

We can try to bin the days birth to ranges maybe according to years so that it could be easier for the model we use

In [ ]:
def get_age_label(days_birth):
    """ Return the age group label (int). """
    age_years = -days_birth / 365
    if age_years < 27: return 1
    elif age_years < 40: return 2
    elif age_years < 50: return 3
    elif age_years < 65: return 4
    elif age_years < 99: return 5
    else: return 0

In [ ]:
df = df[df['CODE_GENDER']!='XNA']

In [ ]:
df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))
df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['CREDIT_TERM'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
df['DAYS_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
    feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
    df[feature_name] = eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)
df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['CREDIT_LENGTH'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

In [ ]:
external_sources = df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
imputer = SimpleImputer(strategy = 'median')
external_sources = imputer.fit_transform(external_sources)
external_sources= pd.DataFrame(external_sources, columns = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH'])

external_sources['SK_ID_CURR'] = df['SK_ID_CURR'].tolist()
df = df.merge(external_sources, on = 'SK_ID_CURR', how = 'left')


In [ ]:
drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'   ,'OWN_CAR_AGE',
'CAR_TO_EMPLOYED_RATIO', 
'CAR_TO_BIRTH_RATIO', 
'EXT_SOURCES_PROD', 
'APARTMENTS_AVG', 
'APARTMENTS_MODE',
'APARTMENTS_MEDI',
'ENTRANCES_AVG', 
'ENTRANCES_MEDI', 
'LIVINGAREA_AVG', 
'FLOORSMAX_MEDI', 
'FLOORSMAX_AVG', 
'FLOORSMAX_MODE', 
'YEARS_BEGINEXPLUATATION_MEDI', 
'TOTALAREA_MODE'
    ]
df.drop(drop_list,axis=1,inplace=True)

# Encoding

In [ ]:
le = LabelEncoder()
for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            # Train on the training data
            le.fit_transform(df[col])
df = pd.get_dummies(df)

In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)

# Modelling

In [ ]:
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
X,y = train.drop(['TARGET','SK_ID_CURR'],axis=1),train['TARGET'].tolist()
print("Train/valid shape: {}, test shape: {}".format(train.shape, test.shape))

In [ ]:
print(X.shape)
print(len(y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=314,stratify=y)

In [ ]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'categorical_feature': 'auto'}

## Learning rate callbacks

In [ ]:
def learning_rate_010_decay_power_099(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_010_decay_power_0995(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.995, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_005_decay_power_099(current_iter):
    base_learning_rate = 0.05
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3


# Set up HyperParameter search space

We use random search, which is more flexible and more efficient than a grid search


In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100


#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)


In [ ]:
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)


In [ ]:
opt_parameters = {'colsample_bytree': 0.9234, 'min_child_samples': 399, 'min_child_weight': 0.1, 'num_leaves': 13, 'reg_alpha': 2, 'reg_lambda': 5, 'subsample': 0.855,'imbalanced':True}

In [ ]:


X_train.columns =  ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
X_test.columns =  ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

clf_final = lgb.LGBMClassifier(**clf.get_params())
#set optimal parameters
clf_final.set_params(**opt_parameters)

#Train the final model with learning rate decay
clf_final.fit(X_train, y_train, **fit_params, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)])

In [ ]:
feat_imp = pd.Series(clf_final.feature_importances_, index=train.drop(['SK_ID_CURR', 'TARGET'], axis=1).columns)
feat_imp.nlargest(20).plot(kind='barh', figsize=(8,10))

## Trying Upsampling the minority class

In [ ]:
# X = X.replace([np.inf, -np.inf], np.nan)
# X.fillna(0, inplace=True)

In [ ]:
# from imblearn.over_sampling import SMOTE
# X_resampled, y_resampled = SMOTE().fit_sample(X, y)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.20, random_state=314,stratify=y_resampled)

In [ ]:
# X_train.columns =  ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
# X_test.columns =  ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]


In [ ]:
# clf_final = lgb.LGBMClassifier(**clf.get_params())
# #set optimal parameters
# clf_final.set_params(**opt_parameters)

# #Train the final model with learning rate decay
# clf_final.fit(X_train, y_train, **fit_params, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)])

In [ ]:
probabilities = clf_final.predict_proba(test.drop(['SK_ID_CURR','TARGET'], axis=1))
submission = pd.DataFrame({
    'SK_ID_CURR': test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
submission.to_csv("submission.csv", index=False)